In [ ]:
%pip install pydantic_settings langchain langchain-core langchain-google-genai langchain-qdrant fastembed langchain-community qdrant-client langgraph

In [ ]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env")

env = Settings()

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_2 = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=env.GOOGLE_API_KEY)

In [ ]:
from qdrant_client.http.models import Distance

collection_name = "mcu_packages"
dimension = 3072
distance = Distance.COSINE

# Create Vector Data

In [ ]:
# load mcu.json data
import json

with open("mcu.json", "r") as f:
    mcu_data = json.load(f)

print(mcu_data[0])

In [ ]:
# Optionally, you can use FastEmbed for embeddings
# from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
# embeddings = FastEmbedEmbeddings(cache_dir="./embedding_cache", model_name="jinaai/jina-embeddings-v2-base-en")
# # https://qdrant.github.io/fastembed/examples/Supported_Models/#supported-text-embedding-models

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

In [ ]:
from qdrant_client.http.models import VectorParams

if(client.collection_exists(collection_name=collection_name) == False):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dimension, distance=distance),
    )

In [ ]:
from qdrant_client.models import PointStruct
import uuid
i = 0
for row in mcu_data:
    i += 1
    text = f"Package Name: {row['name']}, Description: {row['description']}"
    emb = embeddings_2.embed_query(text)
    print(i)
    client.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=str(uuid.uuid4()),  # Generate a unique ID for each point
                vector=emb, 
                payload={
                    "page_content": text,
                    "metadata": {
                            "id": row['id'],
                            "name": row['name'],
                            "description": row['description'],
                    },
                },
            )
        ],
    )
    print(text)

# Create Tool

In [ ]:
from langchain_qdrant import QdrantVectorStore
def get_retriever():

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings_2,
    )
    
    return vector_store.as_retriever()

In [ ]:
from langchain_core.tools import tool
from typing import Annotated, List

@tool
def search_mcu_packages(query: Annotated[str, "search query must contain keywords related to MCU packages"]) -> List[str]:
    """Search for MCU packages by name or description."""
    retriever = get_retriever()
    results = retriever.invoke(query, k=10)
    return [result.page_content for result in results]

In [ ]:
search_mcu_packages("gula darah, diabetes, paket medical check up, siloam")

# Create Agent

In [ ]:
# access the Google Gemini API
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=env.GOOGLE_API_KEY,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides information about Siloam hospitals."),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [ ]:
chain.invoke({"question": "Saya mau cek gula darah di siloam, ada paket apa aja ya ?"})

# Workflow for agent to use tool

In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
# Define state for application
class State(TypedDict):
    question: str
    context: List[str]
    search: str
    answer: str

In [ ]:
def get_context(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
                You are an expert in Medical Check-Up (MCU) packages.
                You will provide keywords about the MCU packages based on the question.
                The keywords should be relevant to the MCU packages available at Siloam hospitals.
                Do not provide any other information.
                If the question already contains keywords, you can return them as is.
                Only return one keyword and in english.
            """),
            ("human", "{question}"),
        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"]})
    return {"search": result.content}

In [ ]:
def retrieve(state: State):
    retrieved_docs = search_mcu_packages(state["search"])
    return {"context": retrieved_docs}

In [ ]:
def generate(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """
                You are an assistant that provides information about Medical Check-Up (MCU) packages at Siloam hospitals.
                You will generate a response based on the context provided.
                The response should be concise and relevant to the question asked.
                package list knowledge: 
                {context}
                If the context is empty, you can provide a general response about MCU packages.
                Please always include related packages in your response.
            """),
            ("human", "{question}"),
        ]
    )
    chain = prompt | llm
    result = chain.invoke({"question": state["question"], "context": state["context"]})
    return {"answer": result.content}

In [ ]:
graph_builder = StateGraph(State).add_sequence([get_context, retrieve, generate])
graph_builder.add_edge(START, "get_context")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({
	"question": "Saya mau cek gula darah di siloam, ada paket apa aja ya ?",
	"context": [],
	"search": "",
	"answer": ""
})
print(response["answer"])